<a href="https://colab.research.google.com/github/A57R4L/LUTModelTraining/blob/main/LUT_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Initialize parameters and import libraries

In [ ]:
## Import libraries

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import numpy as np
import matplotlib.pyplot as pp
import random

# For downloading from Google Colab
import os
from os import path

# For execution time metrics
import time
from datetime import timedelta

# For image tests
from PIL import Image
from skimage import util

# Try to use GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Notebook running on:', device)

# Default full parameters

# Number of Grading Parameters to include in the training
nGradingParameters = 10
# Number of observations/samples - how much training material we create
nSamples = 1024
# Hidden Layer Input size, For 1-2 parameters 10 was enough, 32 for 5, 48 for 10
nHidden = 48
# Batch size, initially 32, tried 64 for 10 parameters
nBatchSize = 32
# Training Steps (epochs)
nTrainSteps = 50000
# Training Seed (Sobol Intialization value, can also be used to invalidate pre-generated training data)
sobolSeed = 1337
# Learnin rate
lRate = 1e-3

# Force sample regeneration (will skip existing sample data)
forceSampleGeneration=False
# Skip model regeneration is one exists in the local folder
skipModelRegeneration=True
# Skip saving local files
skipSave=False
# Download (if not running locally, ie. Google Colab)
DOWNLOAD=False
## For requesting upload of external data to Google Colab - will be ignored if local data exists
UPLOADSAMPLES=False

try:
 from google.colab import files
except:
 print('Not in Google Colab. Disabling download and upload request')
 DOWNLOAD=False
 UPLOADSAMPLES=False

# Demo preset
DEMO=''

# Number of tests for the plot
nTests = 100

Notebook running on: cuda:0


In [ ]:
# Presets for trying notebook with various presets
#
# Fast - 1 grading parameter (Saturation) - 128 samples - 10 hidden layers - 2000 epochs
# Medium - 2 grading parameters (Sat + Contrast) - 256 samples - 16 hidden layers - 3000 epochs
# Slow - 5 grading parameters (Full Sat + Contrast) - 512 samples - 32 hidden layers - 15000 epochs
# Full - 10 grading parameters (Full grading) - 1024 samples - 48 hidden layers - 50000 epochs

DEMO='All1'
# DEMO='Medium'
# DEMO='Slow'
# DEMO='Full'

## Assign preset values
preset=False

if DEMO == 'Fast':
  nGradingParameters = 1
  nSamples = 128
  nHidden = 10
  nTrainSteps = 2000
  lRate = 1e-4
  preset=True

if DEMO == 'Medium':
  nGradingParameters = 2
  nSamples = 256
  nHidden = 16
  nTrainSteps = 3000
  lRate = 1e-4
  preset=True

if DEMO == 'Slow':
  nGradingParameters = 5
  nSamples = 512
  nHidden = 16
  nTrainSteps = 15000
  preset=True

if DEMO == 'Slow2':
  nGradingParameters = 5
  nSamples = 2048
  nHidden = 48
  nTrainSteps = 15000
  nBatchSize = 64
  preset=True

if DEMO == 'Full':
  nGradingParameters = 10
  nSamples = 1024
  nHidden = 48
  nTrainSteps = 50000
  nTests = 256
  preset=True

# Semi consistent results already except Gain W
if DEMO == 'Full2':
  nGradingParameters = 10
  nSamples = 2048
  nHidden = 128
  nTrainSteps = 25000
  nTests = 256
  nBatchSize = 64
  lRate = 1e-3
  preset=True

# No W multipliers, just direct RGB
if DEMO == 'All1':
  nGradingParameters = 12
  nSamples = 4096
  nHidden = 128
  nTrainSteps = 5000
  nTests = 128
  nBatchSize = 64
  lRate = 1e-3
  preset=True

if preset == True:
  print('Running notebook with preset:', DEMO)
else:
  print('Running notebook with Custom Settings')

print('Colorgrading parameters to analyse:', nGradingParameters)
print('LUT samples to generate:', nSamples)
print('Hidden layer inputs:', nHidden)
print('Training steps for NN training:', nTrainSteps)
print('Batch size:', nBatchSize)
print('Learning rate: ', lRate)
print('Test size:', nTests)

Running notebook with preset: All1
Colorgrading parameters to analyse: 12
LUT samples to generate: 4096
Hidden layer inputs: 128
Training steps for NN training: 5000
Batch size: 64
Learning rate:  0.001
Test size: 128


## Define Colorgrading Functions

In [ ]:
# Helper functions and handpicked definitions for data creation
# Grading functions are applied component-wise, implemented here per scalar for the sake of clarity

# Parameters are hardcoded in the start of this cell and here in functions initParameters and grade (unpacking parameters)

# Size of a dimension of the Look-up Texture (16 x 16 x 16)
LUT_SIZE = 16

# Saturation
P_SAT = 1.0
P_SAT_LOW = 0.50
P_SAT_HIGH = 1.50

P_SAT2_LOW = 0.50
P_SAT2_HIGH = 1.50

P_SAT_R = 1.0
P_SAT_G = 1.0
P_SAT_B = 1.0
P_SAT_R_LOW = P_SAT2_LOW
P_SAT_G_LOW = P_SAT2_LOW
P_SAT_B_LOW = P_SAT2_LOW
P_SAT_R_HIGH = P_SAT2_HIGH
P_SAT_G_HIGH = P_SAT2_HIGH
P_SAT_B_HIGH = P_SAT2_HIGH

# Contrast
P_CON = 1.0
P_CON_LOW = 0.50
P_CON_HIGH = 1.50

P_CON2_LOW = 0.50
P_CON2_HIGH = 1.50

P_CON_R = 1.0
P_CON_G = 1.0
P_CON_B = 1.0
P_CON_R_LOW = P_CON2_LOW
P_CON_G_LOW = P_CON2_LOW
P_CON_B_LOW = P_CON2_LOW
P_CON_R_HIGH = P_CON2_HIGH
P_CON_G_HIGH = P_CON2_HIGH
P_CON_B_HIGH = P_CON2_HIGH

# Gamma
P_GAM = 1.0
P_GAM_LOW = 0.50
P_GAM_HIGH = 1.50

P_GAM2_LOW = 0.50
P_GAM2_HIGH = 1.50

P_GAM_R = 1.0
P_GAM_G = 1.0
P_GAM_B = 1.0
P_GAM_R_LOW = P_GAM2_LOW
P_GAM_G_LOW = P_GAM2_LOW
P_GAM_B_LOW = P_GAM2_LOW
P_GAM_R_HIGH = P_GAM2_HIGH
P_GAM_G_HIGH = P_GAM2_HIGH
P_GAM_B_HIGH = P_GAM2_HIGH

# Gain
P_GIN = 1.0
P_GIN_LOW = 0.50
P_GIN_HIGH = 1.50

P_GIN2_LOW = 0.50
P_GIN2_HIGH = 1.50

P_GIN_R = 1.0
P_GIN_G = 1.0
P_GIN_B = 1.0
P_GIN_R_LOW = P_GIN2_LOW
P_GIN_G_LOW = P_GIN2_LOW
P_GIN_B_LOW = P_GIN2_LOW
P_GIN_R_HIGH = P_GIN2_HIGH
P_GIN_G_HIGH = P_GIN2_HIGH
P_GIN_B_HIGH = P_GIN2_HIGH

if nGradingParameters < 12:
  P_ALL_LOW = np.array([P_SAT_LOW, P_CON_LOW, P_SAT_R_LOW, P_SAT_G_LOW, P_SAT_B_LOW, P_GAM_LOW, P_GIN_LOW, P_GIN_R_LOW, P_GIN_G_LOW, P_GIN_B_LOW])
  P_ALL_HIGH = np.array([P_SAT_HIGH, P_CON_HIGH, P_SAT_R_HIGH, P_SAT_G_HIGH, P_SAT_B_HIGH, P_GAM_HIGH, P_GIN_HIGH, P_GIN_R_HIGH, P_GIN_G_HIGH, P_GIN_B_HIGH])
else:
  P_ALL_LOW = np.array([P_SAT_R_LOW, P_SAT_G_LOW, P_SAT_B_LOW, P_CON_R_LOW, P_CON_G_LOW, P_CON_B_LOW, P_GAM_R_LOW, P_GAM_G_LOW, P_GAM_B_LOW, P_GIN_R_LOW, P_GIN_G_LOW, P_GIN_B_LOW])
  P_ALL_HIGH = np.array([P_SAT_R_HIGH, P_SAT_G_HIGH, P_SAT_B_HIGH, P_CON_R_HIGH, P_CON_G_HIGH, P_CON_B_HIGH, P_GAM_R_HIGH, P_GAM_G_HIGH, P_GAM_B_HIGH, P_GIN_R_HIGH, P_GIN_G_HIGH, P_GIN_B_HIGH])

# Parameter list
def initParameters():
  if nGradingParameters < 2:
    paramlist = np.array([P_SAT])
    return paramlist
  if nGradingParameters < 3:
    paramlist = np.array([P_SAT, P_CON])
    return paramlist
  if nGradingParameters < 6:
    paramlist = np.array([P_SAT, P_CON, P_SAT_R, P_SAT_G, P_SAT_B])
    return paramlist
  if nGradingParameters < 12:
    paramlist = np.array([P_SAT, P_CON, P_SAT_R, P_SAT_G, P_SAT_B, P_GAM, P_GIN, P_GIN_R, P_GIN_G, P_GIN_B])
    return paramlist
  paramlist = np.array([P_SAT_R, P_SAT_G, P_SAT_B, P_CON_R, P_CON_G, P_CON_B, P_GAM_R, P_GAM_G, P_GAM_B, P_GIN_R, P_GIN_G, P_GIN_B])
  return paramlist

# Full default
def initFullParameters():
  if nGradingParameters == 12:
    paramlist = np.array([P_SAT_R, P_SAT_G, P_SAT_B, P_CON_R, P_CON_G, P_CON_B, P_GAM_R, P_GAM_G, P_GAM_B, P_GIN_R, P_GIN_G, P_GIN_B])
  else:
    paramlist = np.array([P_SAT, P_CON, P_SAT_R, P_SAT_G, P_SAT_B, P_GAM, P_GIN, P_GIN_R, P_GIN_G, P_GIN_B])
  return paramlist

# Random parameters (random generated (for plotting), even distribution quasirandom (for generation) method defined later)
def randomParameters():
  rndparam = initParameters()
  for i in np.ndindex(rndparam.shape):
    rndparam[i] = random.uniform(P_ALL_LOW[i], P_ALL_HIGH[i])
  return rndparam

# String from parameters - for creating output filenames and printing results
def createParameterString(px, py):
    pStr = f'CUSTOM{px[0]:.2f}v{py[0]:.2f}'
    match nGradingParameters:
      case (1):
        pStr = f'SAT{px[0]:.2f}v{py[0]:.2f}'
      case (2):
        pStr = f'SAT{px[0]:.2f}v{py[0]:.2f}_CON{px[1]:.2f}v{py[1]:.2f}'
      case (5):
        pStr = f'SAT{px[0]:.2f}v{py[0]:.2f}_CON{px[1]:.2f}v{py[1]:.2f}_SATR{px[2]:.2f}v{py[2]:.2f}_SATG{px[3]:.2f}v{py[3]:.2f}_SATB{px[4]:.2f}v{py[4]:.2f}'
      case (10):
        pStr = f'SAT{px[0]:.2f}v{py[0]:.2f}_CON{px[1]:.2f}v{py[1]:.2f}_GAM{px[5]:.2f}v{py[5]:.2f}_GIN{px[6]:.2f}v{py[6]:.2f}'
      case (12):
        pStr = f'SAT{px[0]:.2f}r{px[1]:.2f}g{px[2]:.2f}bx{py[0]:.2f}r{py[1]:.2f}g{py[2]:.2f}b_CON{px[3]:.2f}r{px[4]:.2f}g{px[5]:.2f}bx{py[3]:.2f}r{py[4]:.2f}g{py[5]:.2f}_GAM{px[6]:.2f}r{px[7]:.2f}g{px[8]:.2f}bx{py[6]:.2f}r{py[7]:.2f}g{py[8]:.2f}_GIN{px[9]:.2f}r{px[10]:.2f}g{px[11]:.2f}bx{py[9]:.2f}r{py[10]:.2f}g{py[11]:.2f}'
    return pStr

# Number of parameters (for model training and benchmarking)
P_DEFAULT = initParameters()
# P_ALL = initParameters()
#P_ALL = np.array([P_SAT, P_CON])
nParameters = P_DEFAULT.size

# Transformations
RGB2Y = [0.2722287168, 0.6740817658, 0.0536895174]

# with CAT02 chromatic adaptation transform
SRGB2ACEScg = [[ 0.61311781, 0.34118200,  0.04578734], [ 0.06993408, 0.91810304, 0.01193278], [ 0.02046299,  0.10676866, 0.87271591]]
ACEScg2SRGB = [[ 1.70488733, -0.62415727, -0.08088677], [-0.12952094,  1.13839933, -0.00877924], [-0.02412706, -0.12462061,  1.14882211]]

# Create default LUT
# LUT is 16 x 16 x 16 projected as 256 x 16 2D Texture
def initLUT():
  default_lut = np.zeros((16,256,3), dtype=np.float32)
  r = 0
  g = 0
  b = 0
  # Lutstep needs to be one less for range of 0..1
  lutstep = 1/(LUT_SIZE -1)

  # Note to self: careful with the channel order/how LUT is saved
  for i in np.ndindex(default_lut.shape[:2]):
    default_lut[i] = [r, g, b]
    r = r + lutstep
    if (r >= 1):
      r = 0
      b = b + lutstep
      if (b >= 1):
        b = 0
        g = g + lutstep

  return default_lut

# 3x3 matrix multiplication for colorspace transform (alpha or >3 components are not handled)
def mult_vector3_matrix3(v, m):
    vOut = [m[0][0] * v[0] + m[0][1] * v[1] + m[0][2] * v[2],
            m[1][0] * v[0] + m[1][1] * v[1] + m[1][2] * v[2],
            m[2][0] * v[0] + m[2][1] * v[1] + m[2][2] * v[2]]
    return vOut

# Linear interpolation between two colors
def lerp(color1, color2, value):
  color_out = [0.0, 0.0, 0.0]
  color_out[0] = color1[0] * (1.0 - value[0]) + color2[0] * value[0]
  color_out[1] = color1[1] * (1.0 - value[1]) + color2[1] * value[1]
  color_out[2] = color1[2] * (1.0 - value[2]) + color2[2] * value[2]

  return color_out

# Luminance value of a pixel
def luma(inputcolor):
  luma = np.dot(RGB2Y, inputcolor)
  return luma

# Per pixel Color Grading Function
def grade(inputcolor, parameters):
  # Unpack parameters for clarity
  if nGradingParameters == 12:
    P_SAT_R = parameters[0]
    P_SAT_G = parameters[1]
    P_SAT_B = parameters[2]
    P_CON_R = parameters[3]
    P_CON_G = parameters[4]
    P_CON_B = parameters[5]
    P_GAM_R = parameters[6]
    P_GAM_G = parameters[7]
    P_GAM_B = parameters[8]
    P_GIN_R = parameters[9]
    P_GIN_G = parameters[10]
    P_GIN_B = parameters[11]
    P_SAT = 1.0
    P_CON = 1.0
    P_GIN = 1.0
    P_GAM = 1.0
  else:
    P_SAT = parameters[0]
    P_CON = parameters[1]
    P_SAT_R = parameters[2]
    P_SAT_G = parameters[3]
    P_SAT_B = parameters[4]
    P_GAM = parameters[5]
    P_GIN = parameters[6]
    P_GIN_R = parameters[7]
    P_GIN_G = parameters[8]
    P_GIN_B = parameters[9]

  ## Change colorspace sRGB > (ACEScg) AP1
  inputcolor = mult_vector3_matrix3(inputcolor, SRGB2ACEScg)

  ## Saturation
  # Calculate luminance
  color_luma = luma(inputcolor)
  color_bw = [color_luma,color_luma,color_luma]
  param_sat = [P_SAT * P_SAT_R, P_SAT * P_SAT_G, P_SAT * P_SAT_B]
  color_sat = lerp(color_bw, inputcolor, param_sat)
  # Saturation output can't have negative values
  color_sat[0] = max(0, color_sat[0])
  color_sat[1] = max(0, color_sat[1])
  color_sat[2] = max(0, color_sat[2])

  ## Contrast
  param_con = [P_CON * P_CON_R, P_CON * P_CON_G, P_CON * P_CON_B]
  color_con = [0.0, 0.0, 0.0]
  color_con[0] = (pow(color_sat[0] * (1.0 / 0.18), param_con[0])) * 0.18
  color_con[1] = (pow(color_sat[1] * (1.0 / 0.18), param_con[1])) * 0.18
  color_con[2] = (pow(color_sat[2] * (1.0 / 0.18), param_con[2])) * 0.18

  ## Gamma
  param_gam = [P_GAM * P_GAM_R, P_GAM * P_GAM_G, P_GAM * P_GAM_B]
  color_gam = [0.0, 0.0, 0.0]
  color_gam[0] = pow(color_con[0], (1.0 / param_gam[0]))
  color_gam[1] = pow(color_con[1], (1.0 / param_gam[1]))
  color_gam[2] = pow(color_con[2], (1.0 / param_gam[2]))

  ## Gain
  param_gin = [P_GIN * P_GIN_R, P_GIN * P_GIN_G, P_GIN * P_GIN_B]
  color_gin = [0.0, 0.0, 0.0]
  color_gin[0] = color_gam[0] * param_gin[0]
  color_gin[1] = color_gam[1] * param_gin[1]
  color_gin[2] = color_gam[2] * param_gin[2]

  # Offset is not applied as it doesn't feel relevant in this context (additive lift for all values)

  ## Change colorspace (ACEScg) AP1 > sRGB
  color_out = mult_vector3_matrix3(color_gin, ACEScg2SRGB)

  return color_out

# Grade call function for any image
def applyGrade(img, parameters):
  # Run grading function always with all parameters
  gradeParams = initFullParameters()
  gradeParams[0] = parameters[0]
  # If not running full training, we don't have all parameters
  if nGradingParameters > 1:
    gradeParams[1] = parameters[1]
  if nGradingParameters > 2:
    gradeParams[2] = parameters[2]
    gradeParams[3] = parameters[3]
    gradeParams[4] = parameters[4]
  if nGradingParameters > 5:
    gradeParams[5] = parameters[5]
    gradeParams[6] = parameters[6]
    gradeParams[7] = parameters[7]
    gradeParams[8] = parameters[8]
    gradeParams[9] = parameters[9]
  if nGradingParameters > 11:
    gradeParams[10] = parameters[10]
    gradeParams[11] = parameters[11]

  for iy, ix, iz in np.ndindex(img.shape):
      pixelCol = img[iy, ix]
      gradedCol = grade(pixelCol, gradeParams)
      img[iy, ix] = gradedCol
  # clamp final colors to 0..1
  np.clip(img, 0.0, 1.0, out=img)


This cell applys the grading to a default LUT texture - for grading implementation debugging purposes

## Generate or Load Training Data: LUT samples & grading parameters

Sample-creation is a n-dimensional problem which we tackle by using quasirandom sequence to generate parameters from their hand-picked range

1024 samples generation took: 2min46s on Google T4 GPU - 1m 32s on Local Ryzen 9 5900x / NVidia 4070Ti 12GB

In [ ]:
# With a few parameters we can easily just walk through the space with equally sized steps
# However, with more parameters, the problem becomes n dimensional and we likely want to
# sample the space in a way that doesn't require exponentially growing amount of samples
#
# We use a quasirandom sequence (Sobol) to fill the n dimensional space

# Dimensions are defined by the amount of parameters
# sobolspace = torch.quasirandom.SobolEngine(nParameters, scramble=True, seed=1337)

sobolSamples = torch.zeros(nParameters)

# Dimension of generated sample pool by number of samples we will generate eventually
# sobolsamples = sobolspace.draw(nSamples, sobolsamples)

def initSpace(inputseed, samples):
  sobolspace = torch.quasirandom.SobolEngine(nParameters, scramble=True, seed=inputseed)
  sobolSamples = sobolspace.draw(samples)
  return sobolSamples

def setParameter(p_low, p_high, id, iteration):
  range = p_high - p_low
  value = sobolSamples[iteration,id] * range
  value += p_low
  return value.detach().numpy()

def getAllParameters(iteration):
  currentParam = initParameters()
  for i in np.ndindex(currentParam.shape):
    currentParam[i] = setParameter(P_ALL_LOW[i], P_ALL_HIGH[i], i, iteration)
  return currentParam


In [ ]:
# Trying to locate/upload pre-generated data
LOCALDATA = False

## Check if we have relevant training data locally
SAMPLESPATH = './training_samples'
LUTSAMPLESFILE = f'{SAMPLESPATH}/allsamples_{nSamples}_{nGradingParameters}_{sobolSeed}.npy'
PARAMETERSFILE = f'{SAMPLESPATH}/allparameters_{nSamples}_{nGradingParameters}_{sobolSeed}.npy'

if path.exists(SAMPLESPATH) == False:
  LOCALDATA = False
  os.mkdir(SAMPLESPATH)
else:
  print('Directory found')
  if path.exists(LUTSAMPLESFILE) == False:
    print(f'File: {LUTSAMPLESFILE} not found')
    LOCALDATA = False
  if path.exists(PARAMETERSFILE) == False:
    print(f'File: {PARAMETERSFILE} not found')
    LOCALDATA = False

if LOCALDATA == False:
  if UPLOADSAMPLES == True:
    files.upload()

if path.exists(LUTSAMPLESFILE) == True and path.exists(PARAMETERSFILE) == True:
  LOCALDATA = True

## Catch error (?)
if LOCALDATA != True and LOCALDATA != False:
  raise SystemExit(0)

## Print status
if LOCALDATA == True:
  lut_data = np.load(LUTSAMPLESFILE)
  print('Using pre-generated sample data from:',LUTSAMPLESFILE)
  parameter_targets = np.load(PARAMETERSFILE)
  print('Using pre-generated parameter data from:',PARAMETERSFILE)

  # Check that the shape match
  DATAOK = False
  if lut_data.shape[0] == nSamples and lut_data.shape[1] == LUT_SIZE and lut_data.shape[2] == LUT_SIZE*LUT_SIZE and lut_data.shape[3] == 3:
    if parameter_targets.shape[0] == nSamples and parameter_targets.shape[1] == nGradingParameters:
      DATAOK = True

  if DATAOK==False:
    LOCALDATA = False
    print('Error validating loaded data')
    raise SystemExit(0)
else:
  print('No pre-generated data found')

if forceSampleGeneration==True:
  LOCALDATA = False


Directory found
File: ./training_samples/allsamples_4096_12_1337.npy not found
File: ./training_samples/allparameters_4096_12_1337.npy not found
No pre-generated data found


In [ ]:
## Generate Training Data

if LOCALDATA != True:
  # Store time
  print(f'Generating training data:{nSamples}')
  tStarttime = time.time()

  # Initialize a default lut (no grading applied)
  default_lut = initLUT()

  # LUT data holds all graded LUT tables and parameter targets are the parameters used for that LUT

  lut_data = []
  parameter_targets = []

  # With 1-2 parameters we could still use even linear space distribution, left here for documentation
  # P_SAT_step = (P_SAT_HIGH - P_SAT_LOW) / (nSamples - 1)
  # P_CON_step = (P_CON_HIGH - P_CON_LOW) / (nSamples - 1)

  # Init Quasirandom sampling
  sobolSamples = initSpace(sobolSeed, nSamples)

  current = 0
  while current < nSamples:
    # Get parametes from sequence generator
    currentParameters = getAllParameters(current)
    # P_SAT = P_SAT_LOW + (current * P_SAT_step)
    # P_CON = P_CON_LOW + (current * P_CON_step)

    lut_data.append(np.copy(default_lut))
    applyGrade(lut_data[current], currentParameters)
    parameter_targets.append(currentParameters)
    #parameter_targets.append([P_SAT, P_CON])

    if (current + 1) % 100 == 0:
      tNow = str(timedelta(seconds=(time.time() - tStarttime))).split(".")[0]
      print('Generated samples %5d/%5d: Elapsed: %s' %
            (current + 1, nSamples, tNow))

    current += 1

  # Training data generation is done
  tEndttime = time.time()
  tDelta = timedelta(seconds=(tEndttime - tStarttime))

  print('Generating training data has finished and took:', str(tDelta))



Generating training data:4096
Generated samples   100/ 4096: Elapsed: 0:00:49
Generated samples   200/ 4096: Elapsed: 0:01:40
Generated samples   300/ 4096: Elapsed: 0:02:29
Generated samples   400/ 4096: Elapsed: 0:03:19
Generated samples   500/ 4096: Elapsed: 0:04:08
Generated samples   600/ 4096: Elapsed: 0:04:56
Generated samples   700/ 4096: Elapsed: 0:05:45
Generated samples   800/ 4096: Elapsed: 0:06:34
Generated samples   900/ 4096: Elapsed: 0:07:23
Generated samples  1000/ 4096: Elapsed: 0:08:12
Generated samples  1100/ 4096: Elapsed: 0:09:02


In [ ]:
if LOCALDATA != True and skipSave != True:
  # Save generated training data locally
  np.save(LUTSAMPLESFILE, lut_data)
  print(f'Saved training samples to: {LUTSAMPLESFILE}')
  np.save(PARAMETERSFILE, parameter_targets)
  print(f'Saved parameter samples to: {PARAMETERSFILE}')


  if DOWNLOAD==True:
    # LUTSamples
    files.download(LUTSAMPLESFILE)
    # Parameters
    files.download(PARAMETERSFILE)


In [ ]:
# For debugging and visualizations purposes, let's display a few random samples

PREVIEW=True

if PREVIEW==True:
  idx = 0
  while idx <= 4:
    rndsample = random.randint(0, nSamples-1)
    pp.imshow(lut_data[rndsample])
    pp.show()
    print('Parameter(s)', parameter_targets[rndsample],'\n')
    idx +=1


Network training part

## Prepare (normalize/transform) generated data for training

In [ ]:
# Trying to locate/upload pre-generated, normalized data
LOCALDATA = False

## Check if we have relevant training data locally
SAMPLESPATH = './training_samples'
LUTSAMPLESFILE = f'{SAMPLESPATH}/N_allsamples_{nSamples}_{nGradingParameters}_{sobolSeed}.npy'
PARAMETERSFILE = f'{SAMPLESPATH}/N_allparameters_{nSamples}_{nGradingParameters}_{sobolSeed}.npy'

if path.exists(SAMPLESPATH) == False:
  LOCALDATA = False
  os.mkdir(SAMPLESPATH)
else:
  print('Directory found')
  if path.exists(LUTSAMPLESFILE) == False:
    print(f'File: {LUTSAMPLESFILE} not found')
    LOCALDATA = False
  if path.exists(PARAMETERSFILE) == False:
    print(f'File: {PARAMETERSFILE} not found')
    LOCALDATA = False

if LOCALDATA == False:
  if UPLOADSAMPLES == True:
    files.upload()

if path.exists(LUTSAMPLESFILE) == True and path.exists(PARAMETERSFILE) == True:
  LOCALDATA = True

## Catch error (?)
if LOCALDATA != True and LOCALDATA != False:
  raise SystemExit(0)

## Print status
if LOCALDATA == True:
  lut_data = np.load(LUTSAMPLESFILE)
  print('Using pre-generated sample data from:',LUTSAMPLESFILE)
  parameter_targets = np.load(PARAMETERSFILE)
  print('Using pre-generated parameter data from:',PARAMETERSFILE)

  # Check that the shape match
  DATAOK = False
  if lut_data.shape[0] == nSamples and lut_data.shape[1] == LUT_SIZE and lut_data.shape[2] == LUT_SIZE*LUT_SIZE and lut_data.shape[3] == 3:
    if parameter_targets.shape[0] == nSamples and parameter_targets.shape[1] == nGradingParameters:
      DATAOK = True

  if DATAOK==False:
    LOCALDATA = False
    print('Error validating loaded data')
    raise SystemExit(0)
else:
  print('No pre-generated data found')

if forceSampleGeneration==True:
  LOCALDATA = False


In [ ]:
# Preview of how packing of LUTS work
dflut = initLUT()
pp.imshow(dflut)
pp.show()
packLUT(dflut)
pp.imshow(dflut)
pp.show()
unpackLUT(dflut)
pp.imshow(dflut)
pp.show()

In [ ]:
# Normalize and transform data

# Adjust parameters to range 0..1
def packParam01(param, low, high):
  packed = (param-low) * (1.0 / (high-low))
  return packed

# Return parameter from range 0..1
def unPackParam01(param, low, high):
  unpacked = (param/(1.0 / (high-low))) + low
  return unpacked

parameter_targets = np.array(parameter_targets)

def packParameters(packParameters, nParam):
  idx = 0
  while idx < nParam:
    pLow = P_ALL_LOW[idx]
    # Multiplier
    mulP = 1.0 / (P_ALL_HIGH[idx] - pLow)
    selectedP = packParameters[:,idx]
    # Offset
    selectedP -= pLow
    # Multiply
    selectedP *= mulP
    idx+=1
  return packParameters

def unpackParameters(unpackParameters, nParam):
  idx = 0
  while idx < nParam:
    pLow = P_ALL_LOW[idx]
    # Multiplier
    mulP = 1.0 / (P_ALL_HIGH[idx] - pLow)
    selectedP = unpackParameters[:,idx]
    # Offset
    selectedP /= mulP
    # Multiply
    selectedP += pLow
    idx+=1
  return unpackParameters

if LOCALDATA != True:
  print(parameter_targets[0])
  parameter_targets = packParameters(parameter_targets, nParameters)
  print(parameter_targets[0])

# This will transfer the LUT table in the format that all color components are 0.5, unless grading parameters have altered them
def packLUT(lut):
#  default_lut = np.zeros((16,256,3), dtype=np.float32)
  r = 0
  g = 0
  b = 0
  lutstep = 1/(LUT_SIZE -1)

  for i in np.ndindex(lut.shape[:2]):
  #  default_lut[i] = [r, g, b]
  # Based on the step, shift each component based on their position

    lut[i][0] = lut[i][0] + (0.5 - r)
    lut[i][1] = lut[i][1] + (0.5 - g)
    lut[i][2] = lut[i][2] + (0.5 - b)

    r = r + lutstep
    if (r >= 1):
      r = 0
      b = b + lutstep
      if (b >= 1):
        b = 0
        g = g + lutstep


def unpackLUT(lut):
  r = 0
  g = 0
  b = 0
  lutstep = 1/(LUT_SIZE -1)

  for i in np.ndindex(lut.shape[:2]):
  #  default_lut[i] = [r, g, b]
  # Based on the step, shift each component based on their position

    lut[i][0] = lut[i][0] - (0.5 - r)
    lut[i][1] = lut[i][1] - (0.5 - g)
    lut[i][2] = lut[i][2] - (0.5 - b)

    r = r + lutstep
    if (r >= 1):
      r = 0
      b = b + lutstep
      if (b >= 1):
        b = 0
        g = g + lutstep

#PREVIEW=True

#if PREVIEW==True:
#  idx = 0
#  while idx <= 4:
#    rndsample = random.randint(0, nSamples-1)
#    pp.imshow(lut_data[rndsample])
#    pp.show()
#    print('Parameter(s)', parameter_targets[rndsample],'\n')

#    lut_data[rndsample]=packLUT(lut_data[rndsample])
#    pp.imshow(lut_data[rndsample])
#    pp.show()

#    lut_data[rndsample]=unpackLUT(lut_data[rndsample])
#    pp.imshow(lut_data[rndsample])
#    pp.show()

#    print('')

#    idx +=1

# Process all LUTs for training
if LOCALDATA != True:
  print(f'Generating training data:{nSamples}')
  tStarttime = time.time()

  idx = 0
  while idx < nSamples:
    packLUT(lut_data[idx])
    idx +=1
    if idx % 100 == 0:
        tNow = str(timedelta(seconds=(time.time() - tStarttime))).split(".")[0]
        print('Processed LUT samples for training %5d/%5d: Elapsed: %s' %
              (idx, nSamples, tNow))

  # LUT processing is done
  tEndttime = time.time()
  tDelta = timedelta(seconds=(tEndttime - tStarttime))

  print('LUT processing done and took:', str(tDelta))



In [ ]:
# Save normalized training data

if LOCALDATA != True and skipSave != True:
  # Save generated training data locally
  np.save(LUTSAMPLESFILE, lut_data)
  print(f'Saved training samples to: {LUTSAMPLESFILE}')
  np.save(PARAMETERSFILE, parameter_targets)
  print(f'Saved parameter samples to: {PARAMETERSFILE}')


  if DOWNLOAD==True:
    # LUTSamples
    files.download(LUTSAMPLESFILE)
    # Parameters
    files.download(PARAMETERSFILE)


## Setup Neural Network Training

In [ ]:
## Network dimension

# LUT is 16 per axis and 3 channels (RGB) per pixel
nInput = LUT_SIZE*LUT_SIZE*LUT_SIZE*3
# Outputs are the grading parameters (amount depends on our used preset/settings)
nOutput = nParameters

# Check if there is existing model

LOCALMODEL = False
MODELPATH = './lut_'+DEMO+'_trained_net.pth'
if skipModelRegeneration == True:
  if path.exists(MODELPATH) == True:
    LCOCALMODEL = True
    print('Using pre-generated model, skipping next parts')


In [ ]:
# Define Dataset

class LUTDataSet(Dataset):

    ''' Init function, data=lut samples, targets=output parameters '''
    def __init__(self, data, targets, transform=None):
        self.data = torch.reshape(data, (len(data), nInput))
        self.targets = torch.reshape(targets, (len(targets), nOutput))
        self.transform = transform

    def __len__(self):
        return(len(self.data))

    def __getitem__(self, idx):
        lut = self.data[idx]
        parameters = self.targets[idx]
        return lut, parameters

# Skip if we have a local version of the model
if LOCALMODEL != True:
  # Dataset takes Tensors - convert Numpy Arrays to Tensors
  tParameters = torch.Tensor(parameter_targets)
  tData = torch.Tensor(lut_data)

  d = LUTDataSet(tData, tParameters)

  # Initalize DataLoader
  train_dataloader = DataLoader(d, batch_size=nBatchSize , shuffle=True)


In [ ]:
## Definition of the NN Module

class MLPcondensed(nn.Module):
    '''
    Multi-layer perceptron for non-linear regression.
    '''
    def __init__(self, nInput, nHidden, nOutput):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(nInput, nHidden),
            nn.ReLU(),
            nn.Linear(nHidden, nHidden),
            nn.ReLU(),
            nn.Linear(nHidden, nHidden),
            nn.ReLU(),
            nn.Linear(nHidden, nOutput)
        )

    def forward(self, x):
        return(self.layers(x))

model = MLPcondensed(nInput, nHidden, nOutput)
model.to(device)


## Run Training

What to expect from Google Colab

1 parameter, 128 samples, 10 hidden, 2 000 steps: 42 seconds (CPU) - Loss: 0.000

2 parameters, 256 samples, 16 hidden, 3 000 steps: 2min 39s (CPU) - Loss: 0.001

2 parameters, 256 samples, 16 hidden, 3 000 steps: 1min 7s (T4 GPU) - Loss: 0.001

5 parameters, 512 samples, 16 hidden, 15 000 steps: 47min (CPU) - Loss: 0.448

5 parameters, 512 samples, 16 hidden, 15 000 steps: 39min (TPU) - Loss: 0.127 (why?)

5 parameters, 512 samples, 16 hidden, 15 000 steps: 11min (T4 GPU) - Loss (1): 0.203, Loss (2): 0.140 (why?)

10 parameters, 1024 samples, 48 hidden, 50 000 steps, Batch size 64: 49 min (T4 GPU) - Loss after 1000 epoch: 0.213 - Loss after 50 000 epoch: 0.036

10 parameters, 1024 samples, 48 hidden, 50 000 steps, Batch size 32 : 67 min (Local 4070Ti) - Loss after 50 000 epoch: 0.077

In [ ]:
# Skip if we have a local version of the model
if LOCALMODEL != True:

  # Store time for statistics
  tStarttime = time.time()

  # Define the loss function and optimizer
  loss_function = nn.MSELoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lRate)

  # Set the model to train mode
  model.train()

  # Run the training loop
  for epoch in range(0, nTrainSteps):

    # Set current loss value
    current_loss = 0.0

    # Iterate over the DataLoader for training data
    for i, data in enumerate(train_dataloader, 0):

      # Data
      inputs, targets = data[0].to(device), data[1].to(device)

      # Training
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = loss_function(outputs, targets)
      loss.backward()
      optimizer.step()

      # Statistics
      current_loss += loss.item()

    if (epoch + 1) % 1000 == 0:
        tNow = str(timedelta(seconds=(time.time() - tStarttime))).split(".")[0]
        print('Loss after epoch %5d/%5d: %.3f - Elapsed: %s' %
              (epoch + 1, nTrainSteps, current_loss, tNow))

        current_loss = 0.0

  # Process is complete.
  tEndttime = time.time()
  tDelta = timedelta(seconds=(tEndttime - tStarttime))

  print('Training process has finished and took:', str(tDelta))

  # Save the model locally
  # PATH = './lut_trained_net.pth'
  torch.save(model.state_dict(), MODELPATH)

## Benchmark model and plot results

> Sisennetty osio



In [ ]:
# Let's test our model

# Create random parameters for a new LUT to be tested with the model
# These parameters should match the training data

## Generate Training Data
test_lut = initLUT()

# For plotting
gen_PARAM = np.zeros(shape=(nTests, nParameters))
net_PARAM = np.zeros(shape=(nTests, nParameters))

# Load model
net = MLPcondensed(nInput, nHidden, nOutput)
net.load_state_dict(torch.load(MODELPATH))

# Set the model to evaluation mode
net.eval()

# Local path for LUTS
if path.exists('./LUTS') == False:
  os.mkdir('./LUTS')
LUTPATH = './LUTS/'

current = 0
while current < nTests:
  # Create a random LUT
  param = randomParameters()
  currentLUT = np.copy(test_lut)
  applyGrade(currentLUT, param)
  LUTforModel = np.copy(currentLUT)

  # We need to pack this
  packLUT(LUTforModel)

  # Run in through the  model
  tData = torch.Tensor(LUTforModel)
  # We are giving just one sample to our model
  tData = torch.reshape(tData, (1, nInput))
  output = net(tData)

  # For plotting
  # We need dedicated copy for plotting purposes
  outputnp = output.detach().numpy()#[0]
  # Model output needs to be unpacked (only one parameter needed to unpack)
  outputnp = unpackParameters(outputnp,nParameters)[0]
  gen_PARAM[current] = param
  net_PARAM[current] = outputnp

  # Preview and store every n:th LUT test
  if (current) % 25 == 0:
     print('Test number: ', current)
     pp.imshow(currentLUT)
     pp.show()
     print('')
     viewStr = createParameterString(outputnp, param)
     print('Parameter output from model vs groundtruth:', viewStr)
     print(' ')
     # Save locally
     SAVEPATH = LUTPATH+viewStr+'.png'
     pp.imsave(SAVEPATH, currentLUT)

  current += 1




In [ ]:
# Create plot

# Plot

pp.rcParams.update({'figure.figsize':(8,8), 'figure.dpi':100})
if nGradingParameters == 12:
  pp.scatter(gen_PARAM[:,0], net_PARAM[:,0], s=4, c='#ffbbbb', label=f'Sat R Cor. = {np.round(np.corrcoef(gen_PARAM[:,0],net_PARAM[:,0])[0,1], 4)}')
  pp.scatter(gen_PARAM[:,1], net_PARAM[:,1], s=4, c='#ff9999', label=f'Sat G Cor. = {np.round(np.corrcoef(gen_PARAM[:,1],net_PARAM[:,1])[0,1], 4)}')
  pp.scatter(gen_PARAM[:,2], net_PARAM[:,2], s=4, c='#ff6666', label=f'Sat B Cor. = {np.round(np.corrcoef(gen_PARAM[:,2],net_PARAM[:,2])[0,1], 4)}')

  pp.scatter(gen_PARAM[:,3], net_PARAM[:,3], s=4, c='#ffbbff', label=f'Con R Cor. = {np.round(np.corrcoef(gen_PARAM[:,3],net_PARAM[:,3])[0,1], 4)}')
  pp.scatter(gen_PARAM[:,4], net_PARAM[:,4], s=4, c='#ff99ff', label=f'Con G Cor. = {np.round(np.corrcoef(gen_PARAM[:,4],net_PARAM[:,4])[0,1], 4)}')
  pp.scatter(gen_PARAM[:,5], net_PARAM[:,5], s=4, c='#ff66ff', label=f'Con B Cor. = {np.round(np.corrcoef(gen_PARAM[:,5],net_PARAM[:,5])[0,1], 4)}')

  pp.scatter(gen_PARAM[:,6], net_PARAM[:,6], s=4, c='#bbffbb', label=f'Gam R Cor. = {np.round(np.corrcoef(gen_PARAM[:,6],net_PARAM[:,6])[0,1], 4)}')
  pp.scatter(gen_PARAM[:,7], net_PARAM[:,7], s=4, c='#99ff99', label=f'Gam G Cor. = {np.round(np.corrcoef(gen_PARAM[:,7],net_PARAM[:,7])[0,1], 4)}')
  pp.scatter(gen_PARAM[:,8], net_PARAM[:,8], s=4, c='#66ff66', label=f'Gam B Cor. = {np.round(np.corrcoef(gen_PARAM[:,8],net_PARAM[:,8])[0,1], 4)}')

  pp.scatter(gen_PARAM[:,9], net_PARAM[:,9], s=4, c='#bbbbff', label=f'Gin R Cor. = {np.round(np.corrcoef(gen_PARAM[:,9],net_PARAM[:,9])[0,1], 4)}')
  pp.scatter(gen_PARAM[:,10], net_PARAM[:,10], s=4, c='#9999ff', label=f'Gin G Cor. = {np.round(np.corrcoef(gen_PARAM[:,10],net_PARAM[:,10])[0,1], 4)}')
  pp.scatter(gen_PARAM[:,11], net_PARAM[:,11], s=4, c='#6666ff', label=f'Gin B Cor. = {np.round(np.corrcoef(gen_PARAM[:,11],net_PARAM[:,11])[0,1], 4)}')
else:
  pp.scatter(gen_PARAM[:,0], net_PARAM[:,0], s=8, c='#ff0000', label=f'Sat W Cor. = {np.round(np.corrcoef(gen_PARAM[:,0],net_PARAM[:,0])[0,1], 4)}')
  if nGradingParameters > 1:
    pp.scatter(gen_PARAM[:,1], net_PARAM[:,1], s=8, c='#0000ff', label=f'Con Cor. = {np.round(np.corrcoef(gen_PARAM[:,1],net_PARAM[:,1])[0,1], 4)}')
  if nGradingParameters > 2:
    pp.scatter(gen_PARAM[:,2], net_PARAM[:,2], s=4, c='#ffbbbb', label=f'Sat R Cor. = {np.round(np.corrcoef(gen_PARAM[:,2],net_PARAM[:,2])[0,1], 4)}')
    pp.scatter(gen_PARAM[:,3], net_PARAM[:,3], s=4, c='#ff9999', label=f'Sat G Cor. = {np.round(np.corrcoef(gen_PARAM[:,3],net_PARAM[:,3])[0,1], 4)}')
    pp.scatter(gen_PARAM[:,4], net_PARAM[:,4], s=4, c='#ff6666', label=f'Sat B Cor. = {np.round(np.corrcoef(gen_PARAM[:,4],net_PARAM[:,4])[0,1], 4)}')
  if nGradingParameters > 5:
    pp.scatter(gen_PARAM[:,5], net_PARAM[:,5], s=8, c='#ff00ff', label=f'Gamma Cor. = {np.round(np.corrcoef(gen_PARAM[:,5],net_PARAM[:,5])[0,1], 4)}')
    pp.scatter(gen_PARAM[:,6], net_PARAM[:,6], s=8, c='#00ff00', label=f'Gain W Cor. = {np.round(np.corrcoef(gen_PARAM[:,6],net_PARAM[:,6])[0,1], 4)}')
    pp.scatter(gen_PARAM[:,7], net_PARAM[:,7], s=8, c='#bbffbb', label=f'Gain R Cor. = {np.round(np.corrcoef(gen_PARAM[:,7],net_PARAM[:,7])[0,1], 4)}')
    pp.scatter(gen_PARAM[:,8], net_PARAM[:,8], s=8, c='#99ffff', label=f'Gain G Cor. = {np.round(np.corrcoef(gen_PARAM[:,8],net_PARAM[:,8])[0,1], 4)}')
    pp.scatter(gen_PARAM[:,9], net_PARAM[:,9], s=8, c='#66ff66', label=f'Gain B Cor. = {np.round(np.corrcoef(gen_PARAM[:,9],net_PARAM[:,9])[0,1], 4)}')
pp.title('Parameter Correlation - Samples: '+str(nSamples)+' Steps: '+str(nTrainSteps)+' Training: '+str(tDelta).split(".")[0])
pp.legend()
pp.show()

## Export and download

In [ ]:
## Export and download data in ONNX format

EXPORT=True

if EXPORT==True:

  %pip install onnx
  %pip install onnxscript

  #PATH = './lut_trained_net.pth'
  EXPORTPATH = './LUTmodel'+DEMO+'.onnx'

  # Load model
  exportModel = MLPcondensed(nInput, nHidden, nOutput)
  exportModel.load_state_dict(torch.load(MODELPATH))

  # Initialize model with default LUT (needed for ONNX export)
  defaultLUT = initLUT()
  tData = torch.Tensor(defaultLUT)
  # We are giving just one sample to our model
  tData = torch.reshape(tData, (1, nInput))

  # We are using older export function as dynamo_export version didn't work in UE (5.3)
  # By the time of writing, both Pytorch dynamo_export
  # And Unreal NNE were experimental, so either one could be the culprit

  onnx = torch.onnx.export(exportModel, tData, EXPORTPATH)
  #onnx = torch.onnx.dynamo_export(exportModel, tData)
  #onnx.save(EXPORTPATH)


In [ ]:
# Download (if not running locally, ie. Google Colab)
if DOWNLOAD==True:

  # Created Onnx model
  files.download(EXPORTPATH)

  # Pack LUT folder for download
  !zip -r ./LUTsamples.zip ./LUTS/
  files.download('./LUTsamples.zip')


## Misc Tests

In [ ]:
# Compare LUT to image and parameters (from LUT) to image
import math

# Define functions
def imgfile2array(img):
  im = util.img_as_float(Image.open(img))
  return im

def IndexXY(r, g, b):
#  x = r # red = x axis, repeating on every 16 patch
  y = g # green = along y axis
  x = r + (b * 16) # blue = multiplied by 16
  return int(x), int(y)

def mix(a, b, c):
    c = c - math.floor(c)
    return (1.0 - c) * a + c * b
    #return a + (b - a) * (c - math.floor(c))

def applyLUT(img, lut):
  #for x in img:
  #  for y in x:
  for iy in range(img.shape[0]):
    for ix in range(img.shape[1]):
      pixelCol = img[iy, ix].astype(float)

      pix_r = pixelCol[0] * 15.0
      pix_g = pixelCol[1] * 15.0
      pix_b = pixelCol[2] * 15.0

      rH = math.ceil(pix_r)
      rL = math.floor(pix_r)
      gH = math.ceil(pix_g)
      gL = math.floor(pix_g)
      bH = math.ceil(pix_b)
      bL = math.floor(pix_b)

      iHx, iHy = IndexXY(rH, gH, bH)
      iLx, iLy = IndexXY(rL, gL, bL)

      ColH = lut[iHy][iHx].astype(float)
      ColL = lut[iLy][iLx].astype(float)

      r = mix(ColL[0], ColH[0], pix_r).astype(float)
      g = mix(ColL[1], ColH[1], pix_g).astype(float)
      b = mix(ColL[2], ColH[2], pix_b).astype(float)

      img[iy, ix] = [r,g,b]
  np.clip(img, 0.0, 1.0, out=img)



In [ ]:
# Create random LUT comparison
nLutTests = 5
testfile = './testfiles/sevillapark_640_8bit_srgb.png'

testimage = imgfile2array(testfile)
testlut = initLUT()

current = 0
while current < nLutTests:

  t_img1 = np.copy(testimage)
  t_img2 = np.copy(testimage)
  t_img3 = np.copy(testimage)
  t_img4 = np.copy(testimage)

  param = randomParameters()

  currentLUT = np.copy(test_lut)
  applyGrade(currentLUT, param)

  LUTforModel = np.copy(currentLUT)
  # We need to pack this
  packLUT(LUTforModel)
  # Run in through the  model
  tData = torch.Tensor(LUTforModel)
  # We are giving just one sample to our model
  tData = torch.reshape(tData, (1, nInput))
  output = net(tData)
  # For plotting
  # We need dedicated copy for plotting purposes
  outputnp = output.detach().numpy()#[0]
  # Model output needs to be unpacked (only one parameter needed to unpack)
  outputnp = unpackParameters(outputnp,nParameters)[0]

  ## Images graded with parameters
  # 1: Groundtruth, 2: From model
  applyGrade(t_img1, param)
  applyGrade(t_img2, outputnp)

  ## Images graded with LUTS
  # 3:
  applyLUT(t_img3, currentLUT)

  modelLUT = np.copy(test_lut)
  applyGrade(modelLUT, outputnp)
  applyLUT(t_img4, modelLUT)

  #viewStr = createParameterString(outputnp, param)
  #print('Parameter output from model vs groundtruth:', viewStr)
  print('Groundtruth parameters:', param)
  print('Model output parameters:', outputnp)

  f, axarr = pp.subplots(1,2)
  axarr[0].imshow(t_img1)
  axarr[1].imshow(t_img2)
  pp.show()
  f, axarr = pp.subplots(1,2)
  axarr[0].imshow(t_img3)
  axarr[1].imshow(t_img4)
  pp.show()
  f, axarr = pp.subplots(1,2)
  axarr[0].imshow(currentLUT)
  axarr[1].imshow(modelLUT)
  pp.show()
  current += 1


In [ ]:
# Test and compare all LUTS in dedicated folder
TESTPATH = './testLUTS/'

testfile = './testfiles/sevillapark_640_8bit_srgb.png'
testimage = imgfile2array(testfile)

if path.exists(TESTPATH) == True:
    print('Test LUTS path found. Iterating over all LUT files')
    for subdir, dirs, files in os.walk(TESTPATH):
        for file in files:
            testlutpath = subdir + os.sep + file

            if testlutpath.endswith(".png") or testlutpath.endswith(".PNG"):

                # Print LUT data
                print (testlutpath)
                testLUT = imgfile2array(testlutpath)
                # Drop alpha channel off
                testLUT = testLUT[:,:,:3]
                print (np.shape(testLUT))
                pp.imshow(testLUT)
                pp.show()

                t_img1 = np.copy(testimage)
                t_img2 = np.copy(testimage)

                applyLUT(t_img1, testLUT)

                LUTforModel = np.copy(testLUT)
                # We need to pack this
                packLUT(LUTforModel)
                # Run in through the  model
                tData = torch.Tensor(LUTforModel)
                # We are giving just one sample to our model
                tData = torch.reshape(tData, (1, nInput))
                output = net(tData)
                # For plotting
                # We need dedicated copy for plotting purposes
                outputnp = output.detach().numpy()#[0]
                # Model output needs to be unpacked (only one parameter needed to unpack)
                outputnp = unpackParameters(outputnp,nParameters)[0]

                # Grade test image with modelled parameters
                applyGrade(t_img2, outputnp)

                print("Image graded with LUT vs Model parameters")
                f, axarr = pp.subplots(1,2)
                axarr[0].imshow(t_img1)
                axarr[1].imshow(t_img2)
                pp.show()
                print(outputnp)

## Misc debug functions

In [ ]:
# Display Graded LUT for debug purposes

DEBUG=False

if DEBUG==True:
  debuglut=initLUT()
  debugparameters=initParameters()

#P_SAT
  debugparameters[0]=1.11318099
#P_CON
  if nGradingParameters > 1:
   debugparameters[1]=0.75725073
#P_SAT_RGB
  if nGradingParameters > 2:
   debugparameters[2]=0.91048408
#    debugparameters[3]=1.28483582
#    debugparameters[4]=0.97269809
  if nGradingParameters > 5:
#P_GAM
   debugparameters[5]=1.30009198
#P_GIN
#     debugparameters[6]=1.22534573
#     debugparameters[7]=0.75790048
#     debugparameters[8]=1.22956824
#     debugparameters[9]=1.2244544

  applyGrade(debuglut, debugparameters)
  pp.imshow(debuglut)
  pp.show()
  print('parameter(s)', debugparameters,'\n')



In [ ]:
# Debug Dataset
DEBUG=False

if DEBUG==True:
  for i, data in enumerate(train_dataloader, 0):
      input, target = data
      print('Input Shape', input.shape)
      print('Target Shape', target.shape)
  #    print('Size', input.Size)
      print('Input 0 Shape', input[0].shape)

      #print("In: ", input)
      #print("Out:", target,"\n")

In [ ]:
# Debug Model
DEBUG=False

if DEBUG==True:
  print (model)
  for param in model.parameters():
   print(param)